<center>

**CIFRADOS POR SUSTITUCIÓN**

</center>

<p align="center">
    <img src="https://logowik.com/content/uploads/images/escudo-de-la-universidad-nacional-de-colombia-20163327.logowik.com.webp" width="400">
</p>

# **♻️Cifrado Homófono💱**

<p align="center">
    <img src="https://www.criptohistoria.es/images/homofono2.jpg" width="700">
</p>

<center>

<div align="justify">

El cifrado homófono es una extensión del cifrado por sustitución monoalfabética diseñada para neutralizar el análisis de frecuencias. En vez de asignar un único símbolo de cifra a cada letra del alfabeto claro, se asigna un conjunto de símbolos (homófonos) a las letras más comunes y menos homófonos a las poco frecuentes. Durante la encriptación, cada vez que aparece una letra $a$ se elige al azar uno de sus homófonos, de modo que la frecuencia individual de los símbolos cifrados tienda a ser uniforme.

---


Sea  

* $A=\{a_{1},\dots ,a_{n}\}$ el alfabeto claro (por ejemplo, $n=26$).  
* $B=\{b_{1},\dots ,b_{m}\}$ el alfabeto cifrado, con $m>n$ símbolos (ejemplo: $m=100$ códigos 00–99).

Definimos una función de asignación

$$
f:A\longrightarrow \mathcal P(B)\setminus\{\varnothing\},
\qquad
a\;\mapsto\;f(a)=\{\,b_{i_{1}},\dots ,b_{i_{k(a)}}\},
$$

donde $k(a)=|\,f(a)\,|$ es el número de homófonos adjudicados a la letra $a$.  
Los subconjuntos $f(a)$ deben ser disjuntos y su unión cubrir $B$:

$$
f(a_{i})\cap f(a_{j})=\varnothing\quad(i\neq j),
\qquad
\bigcup_{a\in A} f(a)=B.
$$

---
**Distribución de homófonos**  

Sea $p(a)$ la probabilidad previa de la letra $a$ en el idioma (por ejemplo, $p(\text{E})\approx 0.12$).  
Se eligen los tamaños $k(a)$ para que

$$
\frac{p(a)}{k(a)}\;\approx\;\text{constante},
$$

de modo que cada símbolo cifrado aparezca con frecuencia aproximada

$$
\theta = \sum_{a\in A}\frac{p(a)}{k(a)} \;=\;\frac{1}{m}.
$$

El histograma de los $m$ símbolos tiende así a ser plano.

---

**Algoritmo de encriptación**  

Para un mensaje $x_1x_2\dots x_L\in A^{L}$:

1. Para cada posición $t$ se toma la letra $x_t=a$.  
2. Se selecciona uniformemente un símbolo $y_t\in f(a)$.  
3. El texto cifrado es $y_1y_2\dots y_L\in B^{L}$.

Formalmente, el cifrado define un canal estocástico

$$
\Pr\bigl[\,y_t=b \mid x_t=a \bigr]
=\begin{cases}
\dfrac{1}{k(a)} & \text{si } b\in f(a),\\[6pt]
0 & \text{en otro caso.}
\end{cases}
$$

El descifrado utiliza el diccionario inverso que es $g : B \to A$, definido por:

<center>
$g(b) = a$, donde $a$ es la única letra tal que $b \in f(a)$
</center>


---

**Propiedades estadísticas**  

La probabilidad de observar un símbolo concreto $b\in B$ es

$$
\Pr[b]=\sum_{a\in A}\Pr[b\,|\,a]\;p(a)
      =\sum_{a:\,b\in f(a)}\frac{p(a)}{k(a)}
      \;\approx\;\frac{1}{m},
$$

de modo que las frecuencias de los símbolos cifrados son aproximadamente uniformes y las tácticas clásicas de emparejar “símbolo más frecuente” con “E” dejan de funcionar.

---

**Entropía de la clave**  

Sea $k(a_i)$ la cantidad de símbolos asignados a $a_i$.  
El número total de tablas posibles es

$$
\frac{m!}{\prod_{i=1}^{n} k(a_i)!},
$$

por lo que la entropía de clave es

$$
H=\log_2\!\left(\frac{m!}{\prod_{i} k(a_i)!}\right)\;\text{bits},
$$

habitualmente muy superior a los $\log_2(n!)$ del monoalfabético simple.

---

**Seguridad y criptoanálisis**  

* Ventaja: el atacante ya no puede basarse en frecuencias unigrama; necesita analizar digrama, trigrama o usar búsquedas heurísticas.  
* Limitación: dado texto suficiente, la redundancia natural del lenguaje aún permite romper el cifrado, especialmente si la misma tabla se reutiliza.

---

**Importante**

El cifrado homófono convierte la sustitución fija “uno‑a‑uno”

$$
A \;\longrightarrow\; B
$$

en una sustitución uno‑a‑muchos, controlada por subconjuntos disjuntos $f(a)$.  
Al equilibrar $k(a)$ según $p(a)$, la secuencia cifrada se aproxima a una fuente uniforme de tamaño $m$, elevando la entropía observable y complicando los ataques de frecuencia sobre textos cortos.


</div>

<center>

**📥Importaciones📦**

In [8]:

!pip install -q ipywidgets

import random, secrets, string, unicodedata, re
from typing import Dict, List
import ipywidgets as wd
from IPython.display import display

**👨‍💻Implementación👩‍💻**

In [9]:

_EN_FREQ = "ETAOINSHRDLCUMWFGYPBVKXQJZ"

def _normalize(text: str) -> str:

    tmp = unicodedata.normalize("NFKD", text)
    return "".join(c for c in tmp if not unicodedata.combining(c)).upper()

def _distribution(total_codes: int = 100) -> Dict[str, int]:

    base = {l: 1 for l in string.ascii_uppercase}
    rest = total_codes - 26
    i = 0
    while rest:
        base[_EN_FREQ[i % 26]] += 1
        rest -= 1
        i += 1
    return base

def generate_table(total_codes: int = 100) -> Dict[str, List[str]]:

    codes = [f"{i:02}" for i in range(total_codes)]
    random.shuffle(codes)
    allocation = _distribution(total_codes)
    table, idx = {}, 0
    for letter in string.ascii_uppercase:
        k = allocation[letter]
        table[letter] = codes[idx:idx + k]
        idx += k
    return table

def table_as_html(tbl: Dict[str, List[str]]) -> str:
    rows = ["<pre style='font-family:monospace'>"]
    for l in string.ascii_uppercase:
        rows.append(f"{l}: {' '.join(tbl[l])}")
    rows.append("</pre>")
    return "".join(rows)


**👨‍💻Implementación👩‍💻**

In [10]:

def encrypt(text: str, tbl: Dict[str, List[str]]) -> str:

    tokens = []
    for ch in text:
        if ch == " ":
            tokens.append("/")
        elif ch.upper() in tbl:
            tokens.append(secrets.choice(tbl[ch.upper()]))
        else:
            tokens.append(ch)
    return " ".join(tokens)

def decrypt(cipher: str, tbl: Dict[str, List[str]]) -> str:

    inv = {code: letter for letter, codes in tbl.items() for code in codes}
    result = []
    for tk in cipher.strip().split():
        if tk == "/":
            result.append(" ")
        elif re.fullmatch(r"\d{2}", tk):
            result.append(inv.get(tk, "�"))
        else:
            result.append(tk)
    return "".join(result)


**👨‍💻Implementación👩‍💻**

In [11]:

table = generate_table()
txt = wd.Textarea(value="HELLO WORLD",description="Texto / Códigos:",layout=wd.Layout(width="100%", height="80px"))
mode = wd.ToggleButtons(options=[("Cifrar", "enc"), ("Descifrar", "dec")],description="Modo:")
btn_new = wd.Button(description="Nueva tabla", button_style="info")
out = wd.Output(layout=wd.Layout(border="1px solid #888", padding="6px",min_height="60px"))
accordion = wd.Accordion(children=[wd.HTML(table_as_html(table))])
accordion.set_title(0, "Tabla de homófonos (clic para ver)")


def refresh_output(*_):
    out.clear_output()
    with out:
        try:
            if mode.value == "enc":
                print(encrypt(txt.value, table))
            else:
                print(decrypt(txt.value, table))
        except Exception as e:
            print("❌ Error:", e)

def regenerate(_):
    global table
    table = generate_table()
    accordion.children = [wd.HTML(table_as_html(table))]
    refresh_output()


txt.observe(refresh_output, names="value")
mode.observe(refresh_output, names="value")
btn_new.on_click(regenerate)

**👨‍💻Implementación👩‍💻**

In [12]:
display(wd.VBox([wd.HTML("<h3 style='margin:0'>Cifrado por homófonos</h3>"),txt, mode, btn_new, out, accordion]))
refresh_output()

<center>

**📥Importaciones📦**

In [13]:

!pip install -q ipywidgets
import random, secrets, string, unicodedata, re
from collections import Counter
from typing import Dict, List
import ipywidgets as wd
from IPython.display import display, HTML

**👨‍💻Implementación👩‍💻**

In [14]:

def normalize(text: str) -> str:
    tmp = unicodedata.normalize("NFKD", text)
    return "".join(c for c in tmp if not unicodedata.combining(c)).upper()

def compute_freqs(text: str) -> Dict[str, float]:
    text = normalize(text)
    total = len([c for c in text if c in string.ascii_uppercase])
    count = Counter(c for c in text if c in string.ascii_uppercase)
    return {l: count.get(l, 0) / total if total > 0 else 0.0 for l in string.ascii_uppercase}

def assign_homophones(freqs: Dict[str, float], m=100) -> Dict[str, int]:
    theta = 1 / m
    alloc = {l: max(1, round(freqs.get(l, 0) / theta)) for l in string.ascii_uppercase}
    total = sum(alloc.values())
    while total > m:
        l = max((k for k in alloc if alloc[k] > 1), key=lambda x: alloc[x])
        alloc[l] -= 1
        total -= 1
    while total < m:
        l = max(freqs, key=lambda x: freqs.get(x, 0))
        alloc[l] += 1
        total += 1
    return alloc

def generate_table(text: str, m=100) -> Dict[str, List[str]]:
    freqs = compute_freqs(text)
    alloc = assign_homophones(freqs, m)
    codes = [f"{i:02}" for i in range(m)]
    random.shuffle(codes)
    table, idx = {}, 0
    for l in string.ascii_uppercase:
        k = alloc[l]
        table[l] = codes[idx:idx + k]
        idx += k
    return table

def encrypt(text: str, tbl: Dict[str, List[str]]) -> str:
    tokens = []
    for ch in text:
        if ch == " ":
            tokens.append("/")
        elif ch.upper() in tbl:
            tokens.append(secrets.choice(tbl[ch.upper()]))
        else:
            tokens.append(ch)
    return " ".join(tokens)

def decrypt(cipher: str, tbl: Dict[str, List[str]]) -> str:
    inv = {code: letter for letter, codes in tbl.items() for code in codes}
    result = []
    for tk in cipher.strip().split():
        if tk == "/":
            result.append(" ")
        elif re.fullmatch(r"\d{2}", tk):
            result.append(inv.get(tk, "�"))
        else:
            result.append(tk)
    return "".join(result)

def table_as_html(tbl: Dict[str, List[str]]) -> str:
    html = ["<div style='font-family:monospace; line-height:1.5; white-space:pre; background:#111; color:#ddd; padding:10px; border-radius:6px;'>"]
    for l in string.ascii_uppercase:
        html.append(f"<b>{l}</b>: {' '.join(tbl[l])}")
    html.append("</div>")
    return "".join(html)


tbl = {}
txt = wd.Textarea(value="HELLO WORLD", layout=wd.Layout(width="100%", height="80px"))
mode = wd.ToggleButtons(options=[("Cifrar", "enc"), ("Descifrar", "dec")], description="Modo:")
btn_gen = wd.Button(description="Nueva tabla", button_style="info")
out = wd.Output(layout=wd.Layout(border="1px solid #888", padding="6px", min_height="60px"))
tbl_area = wd.Output()
base_txt = wd.Textarea(value="ESTE ES UN TEXTO DE BASE PARA MEDIR FRECUENCIAS", description="Texto base:", layout=wd.Layout(width="100%", height="60px"))

def refresh_output(*_):
    out.clear_output()
    with out:
        try:
            if mode.value == "enc":
                print(encrypt(txt.value, tbl))
            else:
                print(decrypt(txt.value, tbl))
        except Exception as e:
            print("❌ Error:", e)

def regenerate(_):
    global tbl
    tbl = generate_table(base_txt.value)
    tbl_area.clear_output()
    with tbl_area:
        display(HTML(table_as_html(tbl)))
    refresh_output()

txt.observe(refresh_output, names="value")
mode.observe(refresh_output, names="value")
btn_gen.on_click(regenerate)

display(wd.HTML("<h3 style='margin:0; color:white;'>🔐 Cifrado por homófonos</h3>"),
        base_txt, txt, mode, btn_gen, out,
        wd.HTML("<br><b style='color:white;'>Tabla de homófonos (clic para ver)</b>"),
        tbl_area)

regenerate(None)


HTML(value="<h3 style='margin:0; color:white;'>🔐 Cifrado por homófonos</h3>")

Textarea(value='ESTE ES UN TEXTO DE BASE PARA MEDIR FRECUENCIAS', description='Texto base:', layout=Layout(hei…

Textarea(value='HELLO WORLD', layout=Layout(height='80px', width='100%'))

ToggleButtons(description='Modo:', options=(('Cifrar', 'enc'), ('Descifrar', 'dec')), value='enc')

Button(button_style='info', description='Nueva tabla', style=ButtonStyle())

Output(layout=Layout(border='1px solid #888', min_height='60px', padding='6px'))

HTML(value="<br><b style='color:white;'>Tabla de homófonos (clic para ver)</b>")

Output()